reference: https://www.geeksforgeeks.org/what-is-the-python-global-interpreter-lock-gil/?ref=gcse <br>

Python has something that no other language has that is a reference counter. With the help of the reference counter, we can count the total number of references that are made internally in python to assign a value to a data object. Due to this counter, we can count the references and when this count reaches to zero the variable or data object will be released automatically. 

In [1]:
import sys

geek_var = "Geek"
print(sys.getrefcount(geek_var))
string_gfg = geek_var
print(sys.getrefcount(string_gfg), sys.getrefcount(geek_var))
# garbage collector의 reference count를 센다.

3
4 4


파이썬은 object reference에 의해서 이름은 같아도 본체가 같은(id가 동일한) 변수를 동시에 <br>
두 개 이상의 thread가 건들면 memory leak 혹은 값의 비일관성을 유발한다. <br>
따라서 우리는 여러 thread를 관통하는 lock들을 만들게 되고 잘못하면 치명적인 deadlock을 유발한다. <br>
그래서 memory leak과 deadlock을 예방하기 위해 단 하나의 lock만을 미리 추가했다. <br>
그것이 GIL(Global interpreter lock)이다.

파이썬의 저수준에는 C/C++로 대체로 구현되어 있다. GIL을 사용하면 하나의 lock만 사용하면 되니 <br>
파이썬에서 구현하기도 쉽고 단일 스레드 작업에서 성능이 향상된다. (thread-safe memory management)<br>


In [4]:
# CPU bound program

import time
from threading import Thread
COUNT = 5000000

def countdown(n: int)-> None:
    while n>0:
        n -=1
    
start = time.time()
countdown(COUNT)
end = time.time()
print(f'Time taken in seconds : {end - start}')

Time taken in seconds : 0.4990673065185547


In [5]:
t1 = Thread(target= countdown, args= (COUNT//2,))
t2 = Thread(target= countdown, args= (COUNT//2, ))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()



end = time.time()
print(f'Time taken in seconds : {end - start}')

Time taken in seconds : 0.6345868110656738


분명 같은 작업량을 수행했으나 성능이 느려졌다. <br>
이것은 GIL이 CPU로 하여금 single-thread만 할당했기 때문이다. <br>
과거 python2가 GIL이 있는 C 확장에 크게 의존했기 때문에 파이썬에 아직도 있는 문제다. <br>
그래서 이를 회피하기 위해서 multiprocessing을 도입한다. 그래서 각 process마다 단일 thread가 돌아가는 방식이다. <br>

In [7]:
import multiprocessing

start = time.time()

p1 = multiprocessing.Process(target = countdown, args = (COUNT//2, ))
p2 = multiprocessing.Process(target = countdown, args = (COUNT//2, ))

p1.start()
p2.start()

p1.join()
p2.join()

end = time.time()
print(f'Time taken in seconds : {end - start}')

Time taken in seconds : 3.6697864532470703


이 또한 더 느려졌거나 그대로일 것이다 multiprocess조차 해결해야 할 문제가 있기 때문이다.